In [1]:
!pip install tokenizers==0.5.2

     |████████████████████████████████| 3.7MB 89kB/s 


In [2]:
!pip install simpletransformers

     |████████████████████████████████| 163kB 41.8MB/s 
     |████████████████████████████████| 573kB 34.7MB/s 
     |████████████████████████████████| 204kB 53.4MB/s 
     |████████████████████████████████| 1.0MB 43.5MB/s 
     |████████████████████████████████| 890kB 61.1MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=3970f438130e5701117f65553048ae8f15f7ad6a27b622349b3117f4f094fc36
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=1988cfcb6f02c9989babdd752e1cbccf9755a295de1e15c1c28e08628508c4c5
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built seqeval sacremoses


In [51]:
!pip install emoji

     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=c589a76705b6e3f1334540dabde68a92889e94c6a4a7f4733b6066bf56b31066
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [52]:
!pip install contractions 

     |████████████████████████████████| 317kB 21.9MB/s 
     |████████████████████████████████| 245kB 44.7MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81693 sha256=4b15f4f4153f59018e99bb9dd5a86738a904e6d14216a233a7a8ffa71bee41b5
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [0]:
import os
import re
import pandas as pd
import numpy as np
import emoji
import contractions
from bs4 import BeautifulSoup
from html import unescape

In [0]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value) # Python
    import numpy as np
    np.random.seed(seed_value) # cpu vars
    import torch
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

In [0]:
random_seed(42)

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/Train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/Test.csv')
sub = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/SampleSubmission.csv')

In [15]:
train['label'][~train['label'].isin([0, -1, 1])] = -1
train = train.dropna()
test['safe_text'] = test['safe_text'].fillna('xxxxxx')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
train['label'] = train['label'] * train['agreement']
train = train[['safe_text','label']]
test = test[['safe_text']]

In [0]:
def clean_text(tweet):
    text = BeautifulSoup(unescape(tweet), 'lxml').text
    text = emoji.demojize(text)
    text = contractions.fix(text)
    text = re.sub(r'<user>', ' ', text)
    text = re.sub(r'<url>', ' ', text)
    text = re.sub('RT[\s]+', ' ', text)
    text = re.sub(r'@[\w]*', ' ', text)
    text = re.sub(r'[^a-zA-Z#]', ' ', text)
    # text = re.sub(r'#[\w]*', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

train['safe_text'] = train['safe_text'].apply(clean_text)
test['safe_text'] = test['safe_text'].apply(clean_text)

In [0]:
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score, log_loss, mean_squared_error

In [58]:
y_pred_total = []

for i in range(5): 
    
    X_train, X_cv =  train_test_split(train, test_size=0.20, random_state=i, stratify=train['label'])
    
    model = ClassificationModel('roberta', 'roberta-base', num_labels=1, use_cuda=True, 
                            args={'fp16': False,
                                  'learning_rate': 4e-5,
                                  'regression': True, 
                                  'overwrite_output_dir': True, 
                                  'num_train_epochs': 3,
                                  'manual_seed': i
                                  }
                            )
    
    model.train_model(X_train)
    scores, model_outputs, wrong_predictions = model.eval_model(X_cv)
    print('RMSE:', np.sqrt(mean_squared_error(X_cv['label'], model_outputs)))
    
    predictions, raw_output = model.predict(test['safe_text'])
    y_pred_total.append(predictions)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.271553

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.080820


Running loss: 0.028200


...

Running loss: 0.324660


Running loss: 0.039047


Running loss: 0.331120



/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



RMSE: 0.45108472631589674


In [59]:
np.mean(y_pred_total, 0)

array([-0.04753539,  0.4571744 ,  0.02310776, ...,  0.29354656,
        0.94817716,  0.21412909], dtype=float32)

In [60]:
y_pred_total

[array([-0.10762585,  0.5932041 ,  0.03758769, ...,  0.49119583,
         0.9280424 ,  0.58860713], dtype=float32),
 array([-0.12464173,  0.02828472,  0.01230056, ...,  0.3742512 ,
         0.9599031 ,  0.00342337], dtype=float32),
 array([-0.28054658,  0.4514475 ,  0.01559238, ...,  0.22381441,
         0.9638959 ,  0.00988078], dtype=float32),
 array([-4.0369853e-04,  6.9086337e-01,  1.4259836e-02, ...,
         3.6038911e-01,  9.6831298e-01,  3.7779883e-01], dtype=float32),
 array([0.27554092, 0.5220723 , 0.03579834, ..., 0.01808228, 0.92073137,
        0.0909353 ], dtype=float32)]

In [0]:
y_pred = np.mean(y_pred_total, 0)

In [62]:
sub['label'] = y_pred
sub.head()

,tweet_id,label
0,00BHHHP1,-0.047535
1,00UNMD0E,0.457174
2,01AXPTJF,0.023108
3,01HOEQJW,0.898330
4,01JUKMAO,0.163571


In [63]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(sub)